In [1]:
from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = '/content/drive/My Drive/BERT_NLP/'

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 8.0 MB/s 
     |████████████████████████████████| 67 kB 2.5 MB/s 
     |████████████████████████████████| 895 kB 19.7 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 6.8 MB 30.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", problem_type="multi_label_classification")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [4]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [5]:
import tensorflow as tf
import pandas as pd


In [6]:
# URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
#                                   origin=URL,
#                                   untar=True,
#                                   cache_dir='.',
#                                   cache_subdir='')

colnames=['LABEL_COLUMN', "tweet_id", 'DATA_COLUMN'] 
tweets_df = pd.read_csv(DATASET_PATH+str('retweet_dataset.csv'), ',', header=None, names=colnames, skiprows=1)
tweets_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,LABEL_COLUMN,tweet_id,DATA_COLUMN
0,negative,1.222226e+18,Deputy White House counsel Pat Philis Senate f...
1,negative,1.236290e+18,Panic buy stock toilet roll continue These sce...
2,negative,1.222258e+18,It 's Super Bowl week Crump use prevent defens...
3,negative,1.236433e+18,Turns Donald Crump potentially expose coronavi...
4,positive,1.233925e+18,Had totally surreal author moment B map N toda...


In [7]:
colnames=['tweet_id', "sentiment", "DATA_COLUMN"] 
test_df = pd.read_csv(DATASET_PATH+str('2points_test.tsv'), '\t', header=None, names=colnames)
test_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,tweet_id,sentiment,DATA_COLUMN
0,6.376667e+17,negative,Sony rewards app is like a lot of 19 y.o femal...
1,6.377084e+17,positive,Finally I get my ps4 back I sent it to Sony ca...
2,6.378075e+17,negative,Not Available
3,6.381419e+17,positive,Not Available
4,6.381593e+17,positive,Not Available


In [8]:
LABELS = {'negative':0, 'neutral':1, 'positive':2}
# print(LABELS['negative'])

def convertLabels(sentiment):
  return LABELS[sentiment]
tweets_df['LABEL_COLUMN'] = tweets_df['LABEL_COLUMN'].apply(lambda label: convertLabels(label))
tweets_df.head()

,LABEL_COLUMN,tweet_id,DATA_COLUMN
0,0,1.222226e+18,Deputy White House counsel Pat Philis Senate f...
1,0,1.236290e+18,Panic buy stock toilet roll continue These sce...
2,0,1.222258e+18,It 's Super Bowl week Crump use prevent defens...
3,0,1.236433e+18,Turns Donald Crump potentially expose coronavi...
4,2,1.233925e+18,Had totally surreal author moment B map N toda...


In [9]:
LABELS = {'negative':0, 'positive':1}
# print(LABELS['neutral'])
def convertLabels(sentiment):
  return LABELS[sentiment]
test_df['LABEL_COLUMN'] = test_df['sentiment'].apply(lambda label: convertLabels(label))
test_df.head()

,tweet_id,sentiment,DATA_COLUMN,LABEL_COLUMN
0,6.376667e+17,negative,Sony rewards app is like a lot of 19 y.o femal...,0
1,6.377084e+17,positive,Finally I get my ps4 back I sent it to Sony ca...,1
2,6.378075e+17,negative,Not Available,0
3,6.381419e+17,positive,Not Available,1
4,6.381593e+17,positive,Not Available,1


In [10]:
test_df.drop(labels=['sentiment', 'tweet_id'], axis=1, inplace=True)
test_df.head()

,DATA_COLUMN,LABEL_COLUMN
0,Sony rewards app is like a lot of 19 y.o femal...,0
1,Finally I get my ps4 back I sent it to Sony ca...,1
2,Not Available,0
3,Not Available,1
4,Not Available,1


In [11]:
selected_columns = tweets_df[["DATA_COLUMN","LABEL_COLUMN"]]
tweets_new_df = selected_columns.copy()

In [12]:
selected_columns = test_df[["DATA_COLUMN","LABEL_COLUMN"]]
test_df_new = selected_columns.copy()

In [13]:
# tweets_new_df = tweets_new_df[tweets_new_df.line_race != 'Not Available']

tweets_new_df = tweets_new_df[(tweets_new_df.DATA_COLUMN != "Not Available")]
test_df_new = test_df_new[(test_df_new.DATA_COLUMN != "Not Available")]


tweets_new_df.head()

,DATA_COLUMN,LABEL_COLUMN
0,Deputy White House counsel Pat Philis Senate f...,0
1,Panic buy stock toilet roll continue These sce...,0
2,It 's Super Bowl week Crump use prevent defens...,0
3,Turns Donald Crump potentially expose coronavi...,0
4,Had totally surreal author moment B map N toda...,2


In [14]:
# # The shutil module offers a number of high-level 
# # operations on files and collections of files.
# import os
# import shutil


# # Create main directory path ("/aclImdb")
# main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# # Create sub directory path ("/aclImdb/train")
# train_dir = os.path.join(main_dir, 'train')
# # Remove unsup folder since this is a supervised learning task
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)
# # View the final train folder
# print(os.listdir(train_dir))

In [15]:
# # We create a training dataset and a validation 
# # dataset from our "aclImdb/train" directory with a 80/20 split.
# train = tf.keras.preprocessing.text_dataset_from_directory(
#     'aclImdb/train', batch_size=30000, validation_split=0.2, 
#     subset='training', seed=123)
# test = tf.keras.preprocessing.text_dataset_from_directory(
#     'aclImdb/train', batch_size=30000, validation_split=0.2, 
#     subset='validation', seed=123)

In [16]:
# for i in train.take(1):
#   train_feat = i[0].numpy()
# #   train_lab = i[1].numpy()

# train = pd.DataFrame([train_feat, train_lab]).T
# train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
# train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
# train.head()

In [17]:
# for j in test.take(1):
#   test_feat = j[0].numpy()
#   test_lab = j[1].numpy()

# test = pd.DataFrame([test_feat, test_lab]).T
# test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
# test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
# test.head()

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [19]:
# shuffle(tweets_new_df)

In [20]:
train_full, test = train_test_split(tweets_new_df, test_size=0.1)
train = train_full
# test = test_df_new
train.shape

(1085, 2)

In [21]:
train.LABEL_COLUMN.value_counts()

0    417
2    344
1    324
Name: LABEL_COLUMN, dtype: int64

In [22]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [23]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [24]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [25]:
a = train_InputExamples

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-8, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=10, validation_data=validation_data)

Epoch 1/10


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
while True:
  pass

In [ ]:
tweets_df.LABEL_COLUMN.value_counts()

In [ ]:
test_df_new.LABEL_COLUMN.value_counts()

In [ ]:
tl = list(test_df_new.DATA_COLUMN)

In [ ]:
pred_sentences = tl

In [ ]:
t = tweets_new_df.DATA_COLUMN[0]

In [ ]:
preds_list = []

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative', 'Neutral', 'Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()

for i in range(len(pred_sentences)):
  preds_list.append([pred_sentences[i], labels[label[i]]])


In [ ]:
tf_outputs

In [ ]:
# @title Hello World
print("Hello, World!")

In [ ]:
preds_list

In [ ]:
model.evaluate(tf_batch)

### Testing Now

In [ ]:

test_df = test_df[(test_df.DATA_COLUMN != "Not Available")]

test_df.head()

In [ ]:
selected_columns = test_df[["DATA_COLUMN","sentiment"]]

test_df_new = selected_columns.copy()
test_df_new.head()

In [ ]:
LABELS = {'negative':0, 'positive':1}
# print(LABELS['neutral'])
def convertLabels(sentiment):
  return LABELS[sentiment]

test_df_new['LABEL_COLUMN'] = test_df_new['sentiment'].apply(lambda label: convertLabels(label))
test_df_new.head()

In [ ]:
test_df_new.drop(labels=['sentiment'], inplace=True, axis=1)
test_df_new.head()


In [ ]:
test_InputExamples = test_df_new.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
test = convert_examples_to_tf_dataset(list(test_InputExamples), tokenizer)
# test = test.shuffle(100).batch(32).repeat(2)

In [ ]:
print(train_data)

In [ ]:
model.evaluate(test)